In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/

In [2]:
import os
import pandas as pd

from src.utils.cost_handling import return_cost_per_run

# Categorize Costs
- Use grouping to categorize costs

In [3]:
from_date = "2024-01-01"
to_date="2025-06-23"
timestamp = "2025-06-23 13:23:56.520433"
path_to_dir_retraining = f"/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/data/{from_date}_to_{to_date}/targeting_and_retraining/"
path_to_retraining_data = f"{path_to_dir_retraining}map_nodes_to_prefect_runs_{from_date}_{to_date}.csv"
path_to_save = f"/Users/karolinegriesbach/Documents/Innkeepr/Git/consumption-based-costs/data/{from_date}_to_{to_date}/costs/"
file_costs = f"final_retraining_costs_{from_date}_{to_date}.csv"

In [4]:
os.makedirs(path_to_save, exist_ok=True)

# Load Data

In [5]:
# loading targeting costs mapping (generated via analysis/3_cost_targeting.ipynb)
df_retraining = pd.read_csv(path_to_retraining_data)
df_retraining = df_retraining[[col for col in df_retraining.columns if "Unnamed" not in col]]
df_retraining.shape

In [6]:
# get cost per run
df_retraining = return_cost_per_run(df_retraining)
df_retraining = df_retraining.sort_values(by=["date","serviceName"])
df_retraining = df_retraining[["date","cloud","Prefect_Deployments","serviceName","tenant","audience","duration","sum_duration_serviceName","total_charge_of_serviceName","part_of_costs_per_run","cost_per_run"]]

In [7]:
vc = df_retraining.groupby(by=["date","serviceName"])["part_of_costs_per_run"].sum()
vc_all = vc.reset_index()
vc = vc_all[vc_all["part_of_costs_per_run"].round()!=1]
vc = vc[vc["serviceName"]!="General Purpose Server-g1.3-EU01"] # is used for agentpools as well, so can be != 1
if len(vc) > 0:
    raise ValueError(f"Some the costs does not match the sum: {vc}")

In [8]:
df_retraining = df_retraining[df_retraining["Prefect_Deployments"]=="retraining"]
df_retraining = df_retraining.reset_index(drop=True)
df_retraining

In [9]:
df_retraining.to_csv(f"{path_to_save}{file_costs}",index=False)

# Cost: Server Runs per Day and serviceName

In [10]:
df_retraining.groupby(by=["date","serviceName","tenant","audience_id",""])